shuffle tunning & partitioning


In [2]:
from pyspark.sql import *
import pyspark
from delta import *
from pyspark.sql.functions import broadcast
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, DoubleType
from pyspark.ml.feature import StringIndexer, OneHotEncoder, VectorAssembler
from pyspark.ml.stat import Correlation
from pyspark.ml import Pipeline
from pyspark.ml.regression import LinearRegression
from pyspark.sql.functions import col, lit, approx_count_distinct, concat, count, sum, when, countDistinct,rank
import plotly.graph_objs as go
import pyspark.pandas as ps
#spark = SparkSession.builder.appName("Join CSV Files").getOrCreate()
#spark = SparkSession.builder.appName("Join CSV Files").master("yarn").getOrCreate()
from pyspark import SparkConf, SparkContext


# Set configuration for SparkSession
builder = SparkSession.builder.appName("MyApp") \
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
    .config("spark.executor.instances", 6) \
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog") \
    .master("yarn")

spark = configure_spark_with_delta_pip(builder).getOrCreate()

:: loading settings :: url = jar:file:/home/ubuntu/.local/lib/python3.8/site-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/ubuntu/.ivy2/cache
The jars for the packages stored in: /home/ubuntu/.ivy2/jars
io.delta#delta-core_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-817de2e9-032d-4ca0-977e-45bea93fd8d6;1.0
	confs: [default]
	found io.delta#delta-core_2.12;2.3.0 in central
	found io.delta#delta-storage;2.3.0 in central
	found org.antlr#antlr4-runtime;4.8 in central
:: resolution report :: resolve 290ms :: artifacts dl 16ms
	:: modules in use:
	io.delta#delta-core_2.12;2.3.0 from central in [default]
	io.delta#delta-storage;2.3.0 from central in [default]
	org.antlr#antlr4-runtime;4.8 from central in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   artifacts   |
	|       conf       | number| search|dwnlded|evicted|| number|dwnlded|
	---------------------------------------------------------------------
	|      default     |   3   |   0   |   

23/04/26 14:32:56 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/04/26 14:33:00 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
23/04/26 14:33:03 WARN Client: Neither spark.yarn.jars nor spark.yarn.archive is set, falling back to uploading libraries under SPARK_HOME.


ERROR:root:KeyboardInterrupt while sending command.
Traceback (most recent call last):
  File "/home/ubuntu/.local/lib/python3.8/site-packages/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
  File "/home/ubuntu/.local/lib/python3.8/site-packages/py4j/clientserver.py", line 511, in send_command
    answer = smart_decode(self.stream.readline()[:-1])
  File "/usr/lib/python3.8/socket.py", line 669, in readinto
    return self._sock.recv_into(b)
KeyboardInterrupt


KeyboardInterrupt: 

In [ ]:
import time
start_time = time.time()

: 

In [ ]:
# specify the paths of the folders containing the CSV files in HDFS
folder1_path = "hdfs:///dataset/output/part*"
folder2_path = "hdfs://namenode:9000/dataset/output2/small*"

# define the custom headers for each folder
folder1_header = "index,user_id,region_name,city_name,cpe_manufacturer_name,cpe_model_name,url_host,cpe_type_cd,cpe_model_os_type,price,date,part_of_day,request_cnt"
folder2_header = "user_id,age,is_male"

# define the schema for the CSV files
folder1_schema = StructType([StructField("index", IntegerType(), True),
                             StructField("user_id", IntegerType(), True),
                             StructField("region_name", StringType(), True),
                             StructField("city_name", StringType(), True),
                             StructField("cpe_manufacturer_name", StringType(), True),
                             StructField("cpe_model_name", StringType(), True),
                             StructField("url_host", StringType(), True),
                             StructField("cpe_type_cd", StringType(), True),
                             StructField("cpe_model_os_type", StringType(), True),
                             StructField("price", DoubleType(), True),
                             StructField("date", StringType(), True),
                             StructField("part_of_day", StringType(), True),
                             StructField("request_cnt", StringType(), True)])

folder2_schema = StructType([StructField("user_id", IntegerType(), True),
                             StructField("age", DoubleType(), True),
                             StructField("is_male", StringType(), True)])

# read all CSV files in folder1 into a DataFrame
folder1_df = spark.read.format("csv").schema(folder1_schema).option("header", "false").load(folder1_path).repartition(100)# + "part*")
folder1_df = folder1_df.toDF(*folder1_header.split(","))

#number of dataframe
#num_partitions = folder1_df.rdd.getNumPartitions()
#print("Default number of partitions: ", num_partitions)

# read all CSV files in folder2 into a DataFrame
folder2_df = spark.read.format("csv").schema(folder2_schema).option("header", "false").load(folder2_path)# + "*.csv")
folder2_df = folder2_df.toDF(*folder2_header.split(","))

: 

In [ ]:
folder1_df.cache()
folder2_df.cache()

: 

In [ ]:
folder1_df.show()

: 

In [ ]:
folder2_df.show()

: 

In [ ]:


# perform a right join of the two DataFrames on the "user_id" column
joined_df = folder1_df.join(folder2_df, on=["user_id"], how="right")

# write the result to a new CSV file in HDFS
#joined_df.write.format("csv").option("header", "true").mode("overwrite").save("hdfs://namenode:9000/dataset/output3")


: 

In [ ]:
joined_df.show()

: 

In [ ]:
'''from pyspark.sql.functions import broadcast

df1 = spark.read.parquet("large_dataset1.parquet")
df2 = spark.read.parquet("small_dataset2.parquet")

# use broadcast join to join df1 and df2
joined_df = df1.join(broadcast(df2), "join_column")
'''

: 

In [ ]:
joined_df1 = folder1_df.join(folder2_df, on=["user_id"], how="right")
joined_df1 = joined_df1.hint("merge")

: 

In [ ]:
joined_df2 = folder1_df.join(folder2_df, on=["user_id"], how="right")
joined_df2 = joined_df2.hint("broadcast")

: 

In [ ]:
from pyspark.sql.functions import sum 
from functools import reduce

def drop_null_rows(df):
    # Calculate the percentage of null values in each row
    num_columns = len(df.columns)
    # Expression to count null values in each column
    null_columns = (col(column).isNull().cast("int") for column in df.columns)

    # Add up null values across all columns
    null_count = reduce(lambda x, y: x + y, null_columns)
    null_percentage = (null_count / num_columns) * 100
    
    # Filter out rows with more than 40% null values
    filtered_df = df.filter(null_percentage <= 40)
    
    return filtered_df

: 

In [ ]:
# Remove rows with more than 40% null values
filtered_df = drop_null_rows(joined_df)
filtered_df.show()

: 

In [ ]:
for colm in joined_df.columns:
    col(colm).isNull().cast("int")

: 

In [ ]:
folder1_df.unpersist()
folder2_df.unpersist()

: 

In [ ]:
from pyspark.sql.functions import regexp_replace, col

# remove unwanted \t in the DataFrame
filtered_df = filtered_df.withColumn("request_cnt", regexp_replace(col("request_cnt"), "\t", ""))
filtered_df = filtered_df.withColumn("is_male", regexp_replace(col("is_male"), "\t", ""))
filtered_df = filtered_df.withColumn("request_cnt", col("request_cnt").cast("integer"))
filtered_df = filtered_df.withColumn("is_male", col("is_male").cast("integer"))



: 

In [ ]:
filtered_df.cache()

: 

In [ ]:
filtered_df.show()

: 

In [ ]:
df=filtered_df

: 

In [ ]:
filtered_df.write.format("delta").option("header", "true").mode("overwrite").save("hdfs://namenode:9000/dataset/output3")

: 

In [ ]:
filtered_df=filtered_df.dropna()

: 

##### Check if there are user_id's using more than one cpe_model_name

In [ ]:

filtered_df.groupBy('user_id').agg(countDistinct('cpe_model_name').alias('distinct_models_count')).orderBy('distinct_models_count', ascending=False).show()

: 

In [ ]:
filtered_df.drop('distinct_models_count')

: 

In [ ]:
filtered_df.groupBy('user_id').agg(approx_count_distinct('cpe_model_name').alias('distinct_models_count')).orderBy('distinct_models_count', ascending=False).show()

: 

In [ ]:
filtered_df.unpersist()

: 

In [ ]:
ml_df=filtered_df.drop_duplicates(['user_id','cpe_model_name']) 

: 

#### Remove unnecessary columns

In [ ]:
ml_df=ml_df.drop('index','cpe_type_cd, cpe_model_os_type','date')

: 

In [ ]:
ml_df=ml_df.withColumn("age_bin", 
                          when(col("age") < 18, "0-17").\
                          when((col("age") >= 18) & (col("age") < 30), "18-29").\
                          when((col("age") >= 30) & (col("age") < 50), "30-49").\
                          when(col("age") >= 50, "50+"))

: 

In [ ]:
ml_df.cache()

: 

In [ ]:
ml_df.show()

: 

#### Sum and count of cpe_models

In [ ]:
def filter_by():
    
    ml_df_with_age_bin = ml_df.withColumn("age_bin", 
                          when(col("age") < 18, "0-17").\
                          when((col("age") >= 18) & (col("age") < 30), "18-29").\
                          when((col("age") >= 30) & (col("age") < 50), "30-49").\
                          when(col("age") >= 50, "50+"))
    
    manufacturer_counts = ml_df_with_age_bin.groupBy('cpe_manufacturer_name','cpe_model_name', 'age_bin').agg(count('*').alias('count')).filter(col("age_bin").isNotNull()).orderBy(['age_bin', 'count'], ascending=[True, False])


    window = Window.partitionBy('age_bin').orderBy(col('count').desc())
    ranked_counts = manufacturer_counts.withColumn('rank', rank().over(window))

    top_models = ranked_counts.filter(col('rank') <= 3).drop('rank')

    total_count = top_models.agg(sum('count')).collect()[0][0]
    return(top_models,total_count)



: 

In [ ]:
def filter_by_optimized():
    
    manufacturer_counts = ml_df.groupBy('cpe_manufacturer_name','cpe_model_name', 'age_bin').agg(count('*').alias('count')).filter(col("age_bin").isNotNull()).orderBy(['age_bin', 'count'], ascending=[True, False])


    window = Window.partitionBy('age_bin').orderBy(col('count').desc())
    ranked_counts = manufacturer_counts.withColumn('rank', rank().over(window))

    top_models = ranked_counts.filter(col('rank') <= 3).drop('rank')

    total_count = top_models.agg(sum('count')).collect()[0][0]
    return(top_models,total_count)

: 

#### PieChart function

In [ ]:
def plotFunction(Iphone_11,Iphone_XR,Honor_10,Galaxy_A51,Total,Title):
    df = ps.DataFrame({'count': [Iphone_11, Iphone_XR, Honor_10, Galaxy_A51, Total-(Iphone_11+Iphone_XR+Honor_10+Galaxy_A51)]},
                  index=['Iphone_11', 'Iphone_XR', 'Honor_10', 'Galaxy_A51', 'Others'])

    data = [go.Pie(labels=df.index.tolist(), values=df['count'].tolist())]

    layout = go.Layout(
        paper_bgcolor='rgba(0,0,0,0)',
        plot_bgcolor='rgba(0,0,0,0)',
        height= 600,
        width= 600,
        title=go.layout.Title(text=Title)
    )

    fig = go.Figure(data=data, layout=layout)
    fig.show()

: 

In [ ]:
overall_market=filter_by_optimized()

: 

In [ ]:
plotFunction(2874,760,1294,260,overall_market[1],Title='Overall Market regardless of age')

: 

#### Do one hot encoding for correlation and ML model

In [ ]:
ml_df.show()

: 

In [ ]:
ml_df.dtypes

: 

In [ ]:

indexer = StringIndexer(inputCols=["region_name","city_name","cpe_manufacturer_name","cpe_model_name"], outputCols=["region_name_indexed","city_name_indexed","cpe_manufacturer_name_indexed","cpe_model_name_indexed"])
encoder = OneHotEncoder(inputCols=["region_name_indexed","city_name_indexed","cpe_manufacturer_name_indexed","cpe_model_name_indexed"]\
                        , outputCols=["region_name_encoded","city_name_encoded","cpe_manufacturer_name_encoded","cpe_model_name_encoded"])
assembler = VectorAssembler(inputCols=["age","region_name_encoded","city_name_encoded","cpe_manufacturer_name_encoded","cpe_model_name_encoded","price","request_cnt","is_male"], outputCol="features")

pipeline = Pipeline(stages=[indexer, encoder, assembler])

transformed_df = pipeline.fit(ml_df).transform(ml_df).select("features")

correlation_matrix = Correlation.corr(transformed_df, "features")

: 

#### Show correlation of age

#### Predict age

In [ ]:

indexer = StringIndexer(inputCols=["region_name","city_name","cpe_manufacturer_name","cpe_model_name"], outputCols=["region_name_indexed","city_name_indexed","cpe_manufacturer_name_indexed","cpe_model_name_indexed"])
encoder = OneHotEncoder(inputCols=["region_name_indexed","city_name_indexed","cpe_manufacturer_name_indexed","cpe_model_name_indexed"]\
                        , outputCols=["region_name_encoded","city_name_encoded","cpe_manufacturer_name_encoded","cpe_model_name_encoded"])
assembler = VectorAssembler(inputCols=["region_name_encoded","city_name_encoded","cpe_manufacturer_name_encoded","cpe_model_name_encoded","price","request_cnt","is_male"], outputCol="features")

pipeline = Pipeline(stages=[indexer, encoder, assembler])

data = pipeline.fit(ml_df).transform(ml_df)

train_data, test_data = data.randomSplit([0.75, 0.25])

lr = LinearRegression(featuresCol="features", labelCol="age",regParam=0.5)

model = lr.fit(train_data)
predictions = model.transform(test_data)


: 

Normal prediction

In [ ]:
predicted_age=predictions.select('user_id','cpe_manufacturer_name','cpe_model_name','age','prediction')

predicted_age.show()

: 

Reccommending products for predict data

In [ ]:
predicted_age.cache()

: 

In [ ]:
ml_df.unpersist()

: 

In [ ]:
overall_market_need=overall_market[0]
overall_market_need.show()

: 

In [ ]:
overall_market_need.cache()

: 

In [ ]:
predicted_age.createOrReplaceTempView("my_table")
overall_market_need.createOrReplaceTempView("my_table1")

predicted_age = spark.sql("""
    SELECT *,
        CASE 
            WHEN age BETWEEN 0 AND 17 THEN '0-17'
            WHEN age BETWEEN 18 AND 29 THEN '18-29'
            WHEN age BETWEEN 30 AND 49 THEN '30-49'
            WHEN age >50 THEN '50+'
            ELSE NULL
        END AS age_bracket
    FROM my_table
""")

: 

In [ ]:
predicted_age.show()

: 

In [ ]:
predicted_age.cache()

: 

In [ ]:

predicted_age = predicted_age.withColumn('age_bracket', (when((col('age').between(0, 17)),'0-17')
                    .when((col('age').between(18, 29)),'18-29')
                    .when((col('age').between(30, 49)), '30-49')
                    .when((col('age') >= 50), '50+')
                    .otherwise(None)))

: 

In [ ]:
predicted_age.show()

: 

In [ ]:
suffix = '_r'
for col_name in overall_market_need.columns:
    new_col_name = col_name + suffix
    overall_market_need = overall_market_need.withColumnRenamed(col_name, new_col_name)


: 

In [ ]:
joined_df.cache()

: 

In [ ]:
# join the two dataframes using 'age_bracket' and 'age_bin' columns
joined_df = predicted_age.join(overall_market_need, (col('age_bracket') == col('age_bin_r')), 'left')
joined_df.show()

: 

In [ ]:
predicted_age.unpersist()
overall_market_need.unpersist()

: 

: 

In [ ]:
overall_market_need.dtypes

: 

In [ ]:
from pyspark.sql.functions import concat_ws, col

joined_df = joined_df.withColumn("combined_col1", concat_ws("", col("user_id"), col("cpe_manufacturer_name"), col("cpe_model_name")))


: 

In [ ]:
from pyspark.sql.functions import concat_ws
joined_df = joined_df.withColumn("combined_col1", concat_ws("", col("user_id"), col("cpe_manufacturer_name"), col("cpe_model_name")))


: 

In [ ]:
joined_df = joined_df.withColumn("combined_col2", concat_ws("", col("user_id"), col("cpe_manufacturer_name_r"), col("cpe_model_name_r")))

: 

In [ ]:
'''from pyspark.sql.functions import concat_ws

df = df.withColumn('full_name', concat_ws(' ', col('first_name'), col('last_name')))
'''

: 

In [ ]:
'''

combined_col1 = concat(
    "user_id", "cpe_manufacturer_name", "cpe_model_name").alias("combined_col1")

joined_df = joined_df.select(combined_col1, *joined_df.columns)

combined_col2 = concat(
        "user_id",  "cpe_manufacturer_name_r", "cpe_model_name_r").alias("combined_col2")

joined_df = joined_df.select(combined_col2, *joined_df.columns)'''


: 

In [ ]:
joined_df = joined_df.filter(joined_df['combined_col1'] != joined_df['combined_col2'])


: 

In [ ]:
joined_df.show()

: 

In [ ]:
joined_df = joined_df.dropDuplicates(subset=["user_id"])
joined_df = joined_df.drop('combined_col1', 'combined_col2')


: 

In [ ]:
# Count the number of rows in the DataFrame
num_rows = joined_df.count()

# Print the number of rows
print("Number of rows: ", num_rows)

: 

In [ ]:
joined_df.show()

: 

In [ ]:
joined_df.write.format("delta").option("header", "true").mode("overwrite").save("hdfs://namenode:9000/dataset/output4")

: 

Recommending product for all data

In [ ]:
filtered_df.dtypes

: 

In [ ]:
joined_df.unpersist()

: 

In [ ]:

df = df.withColumn('age_bracket', (when((col('age').between(0, 17)),'0-17')
                    .when((col('age').between(18, 29)),'18-29')
                    .when((col('age').between(30, 49)), '30-49')
                    .when((col('age') >= 50), '50+')
                    .otherwise(None)))
# join the two dataframes using 'age_bracket' and 'age_bin' columns
df = df.join(overall_market_need, (col('age_bracket') == col('age_bin_r')), 'left')
df = df.withColumn("combined_col1", concat_ws("", col("index"), col("cpe_manufacturer_name"), col("cpe_model_name")))
df = df.withColumn("combined_col2", concat_ws("", col("index"), col("cpe_manufacturer_name_r"), col("cpe_model_name_r")))


'''combined_col1 = concat(    "index", "cpe_manufacturer_name", "cpe_model_name").alias("combined_col1")

df = df.select(combined_col1, *df.columns)

combined_col2 = concat(
        "index",  "cpe_manufacturer_name_r", "cpe_model_name_r").alias("combined_col2")

df = df.select(combined_col2, *df.columns)'''

df = df.filter(df['combined_col1'] != df['combined_col2'])
df = df.dropDuplicates(subset=["index"])
df = df.drop('combined_col1', 'combined_col2','user_id','count_r')
df.show()

: 

In [ ]:
df.write.format("delta").option("header", "true").mode("overwrite").save("hdfs://namenode:9000/dataset/output5")

: 

In [ ]:
# Record the end time
end_time = time.time()

# Calculate the total execution time
total_time = end_time - start_time

# Print the execution time
print(f"Execution time: {total_time:.2f} seconds")

# Stop the SparkSession
spark.stop()

: 